# **DEEP LEARNING COURSE PROJECT (IMAGE CLASSIFICATION)**

### **NAME:** ANISH SRIRAM B S
### **SCHOOL:** SCDS
### **EMAIL:** anishsriram.b-26@scds.saiuniversity.edu.in


## TRANSFER LEARNING - XCEPTION MODEL

### DATA PROCESSING

### Details:

*   Total Number of Images in the Dataset: 1661
*   Number of Images present per class:
    *   Number of Images in Motorbikes: 500
    *   Number of Images in Airplanes: 500
    *   Number of Images in Schooner: 63
*   Total Number of Images used for Training and Testing: 1063
    *   Number of Images used for Training: 797
    *   Number of Images used for Testing: 266












In [1]:
# Importing necessary modules
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import glob
import PIL
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from numpy import save

In [2]:
# Unzipping the dataset
!unzip /content/Airplanes_Motorbikes_Schooners.zip

Archive:  /content/Airplanes_Motorbikes_Schooners.zip
   creating: Airplanes_Motorbikes_Schooners/
   creating: Airplanes_Motorbikes_Schooners/Motorbikes/
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0001.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0002.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0003.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0004.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0005.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0006.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0007.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0008.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0009.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0010.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbikes/image_0011.jpg  
  inflating: Airplanes_Motorbikes_Schooners/Motorbike

In [3]:
X = []
y = []

# Preparing the dataset
motorbike_images = glob.glob('/content/Airplanes_Motorbikes_Schooners/Motorbikes/*.jpg')[:500] # Using only 500 Motorbike images
airplane_images = glob.glob('/content/Airplanes_Motorbikes_Schooners/airplanes/*.jpg')[:500] # Using only 500 Airplane images
schooner_images = glob.glob('/content/Airplanes_Motorbikes_Schooners/schooner/*.jpg')
image_files = motorbike_images + airplane_images + schooner_images

# Populating X and y
for item in image_files:
  X_image = Image.open(item)
  X_image = X_image.resize((299,299)) # Resizing the image
  X_image = X_image.convert('RGB') # Ensuring all images are converted to RGB
  X_image = np.array(X_image)/255.0 # Normalizing to range 0.0 to 1.0
  X.append(X_image)
  label = item.split("/")[-2] # Extracting the label from filepath
  y.append(label)


In [4]:
# Using Label Encoder to convert non numeric label to numeric label
LE = LabelEncoder()
y = LE.fit_transform(y)
print("Labels:",LE.classes_)
label_map = {idx: lbl for idx, lbl in enumerate(LE.classes_)}
print("Label Map:",label_map)
X = np.array(X)
y = np.array(y)
print("X_shape:",X.shape)
print("y_shape:",y.shape)


Labels: ['Motorbikes' 'airplanes' 'schooner']
Label Map: {0: 'Motorbikes', 1: 'airplanes', 2: 'schooner'}
X_shape: (1063, 299, 299, 3)
y_shape: (1063,)


In [5]:
# Splitting the dataset into 75% training and 25% testing
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,stratify=y,random_state=42)
print("X_train_shape:",X_train.shape)
print("X_test_shape:",X_test.shape)
print("y_train_shape:",y_train.shape)
print("y_test_shape:",y_test.shape)

# Standardizing using mean and standard deviation
mean = X_train.mean()
std = X_train.std()
X_train_std = (X_train - mean)/std
X_test_std = (X_test - mean)/std


X_train_shape: (797, 299, 299, 3)
X_test_shape: (266, 299, 299, 3)
y_train_shape: (797,)
y_test_shape: (266,)


### TRANSFER LEARNING

In [6]:
# Loading the feature extractor part of the Xception model
base_model = keras.applications.Xception(weights='imagenet',include_top=False)

# Setting the weights of the feature extractor part as non trainable
for layer in base_model.layers:
  layer.trainable = False

# Defining the classifier part
global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output) # Performing Global Average Pooling
x_batchnorm = keras.layers.BatchNormalization()(global_avg_pooling) # Performing Batch Normalization
x_dropout = keras.layers.Dropout(0.25)(x_batchnorm) # Performing Dropout with 25% drop rate
output_ = keras.layers.Dense(units=3,activation="softmax")(x_dropout) # Output layer

model_1_TL = keras.models.Model(inputs=[base_model.input],outputs=[output_])



83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [7]:
# Compiling the model
model_1_TL.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

# Saving the best model using Callbacks
callbacks_1_TL = [keras.callbacks.ModelCheckpoint("Best_model_1_TL.weights.h5",monitor='val_accuracy',save_weights_only=True,save_best_only=True)]

# Training the model
train_model_1_TL = model_1_TL.fit(x=X_train_std,y=y_train,epochs=10,validation_split=0.1,batch_size=16,callbacks=callbacks_1_TL)


Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 36s 428ms/step - accuracy: 0.8652 - loss: 0.3387 - val_accuracy: 1.0000 - val_loss: 0.1170
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 15s 116ms/step - accuracy: 0.9981 - loss: 0.0057 - val_accuracy: 1.0000 - val_loss: 0.0674
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.9973 - loss: 0.0063 - val_accuracy: 1.0000 - val_loss: 0.0371
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.9994 - loss: 0.0049 - val_accuracy: 1.0000 - val_loss: 0.0189
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 11s 120ms/step - accuracy: 1.0000 - loss: 8.3605e-04 - val_accuracy: 1.0000 - val_loss: 0.0093
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 119ms/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 0.0039
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 117ms/step - accuracy: 0.9960 - loss: 0.0074 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 10s 120ms/step - accuracy: 0.9994 - loss: 0.0046 - val_ac

### SAVING THE MODEL AND DATASET

In [8]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [9]:
# Saving the best model obtained
model_1_TL.save('/content/drive/MyDrive/Deep Learning Project/Model1/Best_model_1_TL.keras')

# Saving the dataset
save('/content/drive/MyDrive/Deep Learning Project/Dataset/X_train_std.npy', X_train_std)
save('/content/drive/MyDrive/Deep Learning Project/Dataset/X_test_std.npy', X_test_std)
save('/content/drive/MyDrive/Deep Learning Project/Dataset/y_train.npy', y_train)
save('/content/drive/MyDrive/Deep Learning Project/Dataset/y_test.npy', y_test)